# Twitter Data Extraction 

Module Code: **CS3PP19**

Assignment Report Title: **Twitter Data Extraction**

Student Numbers: **28010336, 28012606, 28006461**

Date Completed: **/11/2021** 

Actual hours spent for the assignment: **2 hours**

Assignment evaluation (3 key points):  
1. 
2. 
3.

# Introduction

In this data analysis task, we will analyse 3 selected Twitter accounts *for a variety of information.*

In task 1 we gather data from twitter regarding 3 user accounts, and process this data into Pandas dataframes along with cleansing them down to crtical data only to reduce the size of our dataset(s).

In task 2, we merge all data from the accounts and perform an explroatory data analysis (EDA) on the conjoined data in order to examine their characteristics. Specifically, we intend to analyse the following:

- Follower to following ratio (including the individual counts of each) plotted as a percentage bar of the ratio
- Activity share (status updates, retweets, favourites) represented as a piechart
- Most popular mentions
- 



# Task 1 - Data Gathering and Pre-processing

## API connection

Using Tweepy, we use our twitter developer account’s credentials to authorise it’s usage and create an instance of the Tweepy API.

https://docs.tweepy.org/en/stable/auth_tutorial.html

In [3]:
import tweepy

# API credentials
api_key = 'eqpYibo0WT2jOr675owJSUNmi'
api_secret_key = 'rh8HHqs79JSAyyp9xWZBV9BVrGxqBqOKHml2oRDU3bTzU8XVcj'

# Access token credentials
access_token = '1449033199422693384-E5rEswGDQSyhGUljpUSTrHK1Vuc5az'
access_token_secret = 'jVjhFidOjMKEQtmRdTVgnkauwYcy45NpSl50WjT5a77bJ'


auth = tweepy.OAuthHandler(api_key, api_secret_key) # Create authorisation instance
auth.set_access_token(access_token, access_token_secret) # Configure with our access token

api = tweepy.API(auth); # Create an instance of the tweepy API using the above authenication instance

Store user account references in a list

https://docs.tweepy.org/en/stable/api.html#tweepy.API.get_user

In [4]:
#users = [api.get_user(screen_name = 'BarackObama'), 
#         api.get_user(screen_name = 'iJustine'), 
#         api.get_user(screen_name = 'LeoDiCaprio')]

#Initialise the dictionary to store raw data
dUsers = {'BarackObama': api.get_user(screen_name = 'BarackObama'),
          'iJustine': api.get_user(screen_name = 'iJustine'),
          'LeoDiCaprio': api.get_user(screen_name = 'LeoDiCaprio')}

## Data extraction

In the data extraction stage, we define and select the data we require for our analysis; for us, we define this as a set of API calls and selecting the raw data from which we will use - reducing the size our dataset(s).

In order to obtain the following list, we searched the api documentation of Tweepy (https://docs.tweepy.org/en/stable/api.html) and sought at methods which can acquire useful data of an account. These methods are:
- Friends - get_friends()
- Followers - get_followers()
- Posts - user_timeline()
- Retweets - get_retweets(), get_retweets_of_me()
- Likes - get_favourites()

The list below shows all the raw data that we will be extracting from the accounts:
- Number of accounts that they are following.
- Number of followers.
- Number of original tweets that were posted.
- Number of tweets that the users have liked.
- Number of likes that each original tweet has.
- Number of times that each original tweet have been retweeted.
- Number of retweets that were posted by the user.
- Number of times that the accounts were mentioned.

In [5]:
import pandas as pd
import json

# https://developer.twitter.com/en/docs/twitter-api/rate-limits

class RawUserData:
    # User information
    sUsername = None # The username of the account this UserData represents
    iDateJoined = None # The date the account was created
    
    # Metrics
    iFriendsCount = None # The number of people this account follows
    iFollowersCount = None # The number of people following this account
    iTweetsCount = None # The number of tweets this account has made
    iLikesCount = None # The number of likes the account has done
    iRetweetsCount = None # The number of retweets the account has done
    iMentionsCount = None # The number of times this account has mentioned someone/something in a tweet
    
    # User activity
    lTweets = [] # The tweets this account has made (original tweets)
    lRetweets = [] # The retweets this account has made
    lLikes = [] # The likes this account has made
    
    # Network
    lMentions = [] # What this account has mentioned
    
    """def __init__(oSelf, oUser):
        
        oSelf.sUsername = oUser.name
        oSelf.iDateJoined = oUser.created_at.strftime('%d-%m-%Y')
        oSelf.iFriendsCount = oUser.friends_count
        oSelf.iFollowersCount = oUser.followers_count
        oSelf.iTweetsCount= oUser.statuses_count
        oSelf.iLikesCount = oUser.favourites_count
        
        print(f"{oSelf.sUsername}:\n"
              f"\t- Data Joined: {oSelf.iDateJoined}\n"
              f"\t- Friends: {oSelf.iFriendsCount}\n"
              f"\t- Followers: {oSelf.iFollowersCount}\n"
              f"\t- Tweets: {oSelf.iTweetsCount}\n"
              f"\t- Likes: {oSelf.iLikesCount}\n")"""
    
    #Changed the name of the function as __init__ did not make sense
    def extractData():
        #Go through each value in the list
        for oUser in dUsers.values():
            #Store all of the raw data into global variables
            sUsername = oUser.name
            iDateJoined = oUser.created_at.strftime('%d-%m-%Y')
            iFriendsCount = oUser.friends_count
            iFollowersCount = oUser.followers_count
            iTweetsCount= oUser.statuses_count
            iLikesCount = oUser.favourites_count
            
            #Print raw data
            print(f"\tUsername: {sUsername}\n"
                  f"\t- Data Joined: {iDateJoined}\n"
                  f"\t- Followers: {iFollowersCount}\n"
                  f"\t- Tweets: {iTweetsCount}\n"
                  f"\t- Likes: {iLikesCount}\n")
    
    #Invoke function here for now
    extractData()   
    
    #CAN USE THIS TO GET LIKES AND RETWEETS
    def ExamplePrintTweets(oSelf):
        iCount = 0
        for oTweet in tweepy.Cursor(api.user_timeline, screen_name=sUsername, tweet_mode="extended").items():
            iCount += 1
            print(f"{iCount}\t{tweet.user.id}\t{oTweet.user.screen_name}\t{oTweet.created_at}\t{oTweet.full_text}")
        
    def ToPandasDataframe(oSelf):
        return
    
    #for friend in tweepy.Cursor(api.get_friends, screen_name = 'iJustine', count = 100).items():"""

	Username: Barack Obama
	- Data Joined: 05-03-2007
	- Followers: 130230307
	- Tweets: 16311
	- Likes: 0

	Username: iJustine
	- Data Joined: 10-10-2006
	- Followers: 1795155
	- Tweets: 60394
	- Likes: 21751

	Username: Leonardo DiCaprio
	- Data Joined: 16-04-2010
	- Followers: 19408517
	- Tweets: 2479
	- Likes: 327



In [6]:
lTweets?

Object `lTweets` not found.


## Data cleaning
*From the selected data, we resolve any missing values, duplicated occurences, etc.*

In preperation for the next stage of pre-processing of the data.

This step is crucial in ensuring the resultant final information extracted is accurate and reliable. Furthermore, cleaning the data extracted allows for pre-processing to be a simpler and more reliable process as duplicate or missing values are resolved.

In [12]:
#Go through each account, in the list, to extract the required data and their raw contents as a UserData object
for oUser in lUsers.values():
    userData = UserData(oUser)
    #userData.ExamplePrintTweets()

NameError: name 'UserData' is not defined

## Data pre-processing
The pre-processing stage of data extraction requires the selected raw data be prepared for analysis. To do so, we transform the cleaned raw data into Pandas data frames which can be done almost directly as the data was cleaned in the previous stage.

For an account:

| User        | Description |
| ----------- | ----------- |
| Header      | Title       |
| Paragraph   | Text        |

| User        | Description |
| ----------- | ----------- |
| Header      | Title       |
| Paragraph   | Text        |


| User        | Description |
| ----------- | ----------- |
| Header      | Title       |
| Paragraph   | Text        |



In [33]:
# Take the RawUserData of an account and transform it into Pandas dataframes

# User information
    sUsername = None # The username of the account this UserData represents
    iDateJoined = None # The date the account was created
    
    # Metrics
    iFriendsCount = None # The number of people this account follows
    iFollowersCount = None # The number of people following this account
    iTweetsCount = None # The number of tweets this account has made
    iLikesCount = None # The number of likes the account has done
    iRetweetsCount = None # The number of retweets the account has done
    iMentionsCount = None # The number of times this account has mentioned someone/something in a tweet
    
    # User activity
    tweets = [] # The tweets this account has made (original tweets)
    retweets = [] # The retweets this account has made
    likes = [] # The likes this account has made
    
    # Network
    mentions = [] # What this account has mentioned

class PandasUserData:
    
    userData = None
    
    # Status updates
    
    
    #def __init__(self, userData):
    def extractData():
        
        

SyntaxError: unexpected EOF while parsing (<ipython-input-33-0d44fc6e10d7>, line 33)

## Task 1 Conclusion

# Task 2 - Exploratory Data Analysis (EDA)